# Project Euler, Problems 1-10

In [23]:
using BenchmarkTools
using Base.Iterators: take

## Problem 1
If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.
Find the sum of all the multiples of 3 or 5 below 1000.

In [24]:
sum(x for x in 1:999 if ((x%3 == 0) || (x%5 ==0)))

233168

## Problem 2
Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:

1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

In [25]:
?Base.IteratorSize

```
IteratorSize(itertype::Type) -> IteratorSize
```

Given the type of an iterator, return one of the following values:

  * `SizeUnknown()` if the length (number of elements) cannot be determined in advance.
  * `HasLength()` if there is a fixed, finite length.
  * `HasShape{N}()` if there is a known length plus a notion of multidimensional shape (as for an array).  In this case `N` should give the number of dimensions, and the [`axes`](@ref) function is valid  for the iterator.
  * `IsInfinite()` if the iterator yields values forever.

The default value (for iterators that do not define this function) is `HasLength()`. This means that most iterators are assumed to implement [`length`](@ref).

This trait is generally used to select between algorithms that pre-allocate space for their result, and algorithms that resize their result incrementally.

```jldoctest
julia> Base.IteratorSize(1:5)
Base.HasShape{1}()

julia> Base.IteratorSize((2,3))
Base.HasLength()
```


In [26]:
struct Fibonacci
    x::Int
    y::Int
end

Base.iterate(iter::Fibonacci) = iter.x, (iter.x, iter.y)
Base.iterate(iter::Fibonacci, state) = state[2], (state[2], state[1]+state[2])
Base.IteratorSize(::Fibonacci) = Base.IsInfinite()

In [27]:
sum(Iterators.filter(x -> iseven(x) && x < Int(4e6), take(Fibonacci(0,1), 50)))

4613732

## Problem 3

The prime factors of 13195 are 5, 7, 13 and 29.

What is the largest prime factor of the number 600851475143 ?

In [28]:
#sieve of Erasthosenes
function PrimeSieve(N::Int64)
    isprime = trues(N)
    @inbounds for i = 2:Int64(trunc(sqrt(N)))
        if isprime[i]
            @inbounds for j in (i*i):i:N
                isprime[j] = false
            end
        end
    end
    return filter(x -> isprime[x], 2:N)
end

PrimeSieve (generic function with 1 method)

In [29]:
function LargestPrimeFactor(N::Int64)
    for x in reverse(PrimeSieve(10000))
        if N % x == 0
            return x
        end
    end
    return N
end 

LargestPrimeFactor (generic function with 1 method)

In [30]:
LargestPrimeFactor(600851475143)

6857

## Problem 4
A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99.

Find the largest palindrome made from the product of two 3-digit numbers.

In [31]:
maximum(filter(x -> reverse("$x") == "$x", [x*y for x in 100:999 for y in x:999]))

906609

## Problem 5

2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder.

What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?

In [32]:
#Euclid's algorithm for GCD
function GCD(a::Int64, b::Int64)
    if b == 0
        return a
    else
        return GCD(b, a % b)
    end
end 
LCM(a::Int64, b::Int64) = div(abs(a), GCD(a,b)) * abs(b)

LCM (generic function with 1 method)

In [33]:
reduce(LCM, 1:20, init=1)

232792560

## Problem 6

Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

In [34]:
sum(1:100)^2 - sum(map(x -> x^2, 1:100)) 

25164150

## Problem 7
What is the 10001st prime number?

In [38]:
function nthprime(N::Int64)
    if N < 2
        return 2
    elseif N == 2
        return 3
    end
    upper = round(Int, N*(log(N) + log(log(N))))
    primes = PrimeSieve(upper)
    return primes[N]
end

nthprime (generic function with 1 method)

In [42]:
nthprime(10001)

104743

## Problem 8

In [44]:
?replace

search: replace replace! redisplay



```
replace(A, old_new::Pair...; [count::Integer])
```

Return a copy of collection `A` where, for each pair `old=>new` in `old_new`, all occurrences of `old` are replaced by `new`. Equality is determined using [`isequal`](@ref). If `count` is specified, then replace at most `count` occurrences in total.

The element type of the result is chosen using promotion (see [`promote_type`](@ref)) based on the element type of `A` and on the types of the `new` values in pairs. If `count` is omitted and the element type of `A` is a `Union`, the element type of the result will not include singleton types which are replaced with values of a different type: for example, `Union{T,Missing}` will become `T` if `missing` is replaced.

See also [`replace!`](@ref).

# Examples

```jldoctest
julia> replace([1, 2, 1, 3], 1=>0, 2=>4, count=2)
4-element Array{Int64,1}:
 0
 4
 1
 3

julia> replace([1, missing], missing=>0)
2-element Array{Int64,1}:
 1
 0
```

---

```
replace(new::Function, A; [count::Integer])
```

Return a copy of `A` where each value `x` in `A` is replaced by `new(x)` If `count` is specified, then replace at most `count` values in total (replacements being defined as `new(x) !== x`).

# Examples

```jldoctest
julia> replace(x -> isodd(x) ? 2x : x, [1, 2, 3, 4])
4-element Array{Int64,1}:
 2
 2
 6
 4

julia> replace(Dict(1=>2, 3=>4)) do kv
           first(kv) < 3 ? first(kv)=>3 : kv
       end
Dict{Int64,Int64} with 2 entries:
  3 => 4
  1 => 3
```

---

```
replace(s::AbstractString, pat=>r; [count::Integer])
```

Search for the given pattern `pat` in `s`, and replace each occurrence with `r`. If `count` is provided, replace at most `count` occurrences. `pat` may be a single character, a vector or a set of characters, a string, or a regular expression. If `r` is a function, each occurrence is replaced with `r(s)` where `s` is the matched substring (when `pat` is a `Regex` or `AbstractString`) or character (when `pat` is an `AbstractChar` or a collection of `AbstractChar`). If `pat` is a regular expression and `r` is a [`SubstitutionString`](@ref), then capture group references in `r` are replaced with the corresponding matched text. To remove instances of `pat` from `string`, set `r` to the empty `String` (`""`).

# Examples

```jldoctest
julia> replace("Python is a programming language.", "Python" => "Julia")
"Julia is a programming language."

julia> replace("The quick foxes run quickly.", "quick" => "slow", count=1)
"The slow foxes run quickly."

julia> replace("The quick foxes run quickly.", "quick" => "", count=1)
"The  foxes run quickly."

julia> replace("The quick foxes run quickly.", r"fox(es)?" => s"bus\1")
"The quick buses run quickly."
```


In [47]:
bigString = replace("""73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450""", '\n'=>"");

In [86]:
maximum([mapreduce(x->parse(Int,x), *, y) 
        for y in [bigString[i:i+12] 
                for i in 1:length(bigString)-12] if !('0' in y)])

23514624000

## Problem 9

In [89]:
[((a,b,c), a*b*c) for a in 1:1000 for b in a:1000 for c in b:1000 if (a^2 + b^2 == c^2 && a+b+c == 1000)]

1-element Array{Tuple{Tuple{Int64,Int64,Int64},Int64},1}:
 ((200, 375, 425), 31875000)

## Problem 10

In [90]:
sum(PrimeSieve(2000000))

142913828922